# Chingu Member Data Cleaning v58

# Project Overview
This project's ultimate aim is to create an app that visualizes demographics for Chingu Members. Further project requirements and specifications set by the Chingu Organization can be found on github here: https://github.com/chingu-voyages/voyage-project-chingu-map
# Our Team: 
- Formed for Voyage 58, we are team 37
- Michael, Shruti, Jessica, Henry, Gisele
- Our github repo containing the solution to the requirements: [V58-tier3-team-37](https://github.com/chingu-voyages/V58-tier3-team-37)
# This Notebook's Goal
- Prepare the raw data so that it's usable for visualizations set by the requirements
	- become familar with the data
	- remove illogical data
	- remove duplicate data
	- separate or normalize the data into logical tables
	- report information on data that was removed
- This notebook contains the steps needed are to clean up the raw data and the reasoning behind them.
# Data Source
The json document was provided on github in the project details here: 
https://github.com/chingu-voyages/voyage-project-chingu-map/blob/main/src/assets/chingu_info.json

# Plan part1
1. Download the data
2. inspect the input format
3. decide on a strategy to flatten the data into tables

In [250]:
import requests
import json
raw_url = 'https://raw.githubusercontent.com/chingu-voyages/voyage-project-chingu-map/main/src/assets/chingu_info.json'
response = requests.get(raw_url)
if response.status_code == 200:
    chingu_info = response.json()
    print(json.dumps(chingu_info[:50], indent=2))
else:
    print(f"Failed to fetch JSON: {response.status_code}")


[
  {
    "Timestamp": "2025-11-05 13:33",
    "Gender": "MALE",
    "Country Code": "NZ",
    "Timezone": "",
    "Goal": "ACCELERATE LEARNING",
    "Goal-Other": "",
    "Source": "PERSONAL NETWORK",
    "Source-Other": "",
    "Country name (from Country)": "New Zealand",
    "Solo Project Tier": "",
    "Role Type": "Web",
    "Voyage Role": "Developer",
    "Voyage (from Voyage Signups)": "",
    "Voyage Tier": ""
  },
  {
    "Timestamp": "2025-11-05 06:52",
    "Gender": "FEMALE",
    "Country Code": "IN",
    "Timezone": "",
    "Goal": "GAIN EXPERIENCE",
    "Goal-Other": "",
    "Source": "GOOGLE SEARCH",
    "Source-Other": "",
    "Country name (from Country)": "India",
    "Solo Project Tier": "",
    "Role Type": "Python",
    "Voyage Role": "Developer",
    "Voyage (from Voyage Signups)": "",
    "Voyage Tier": ""
  },
  {
    "Timestamp": "2025-11-04 09:14",
    "Gender": "MALE",
    "Country Code": "GE",
    "Timezone": "",
    "Goal": "NETWORK WITH SHARED GOALS",
    

# 1st impressions
This appears to be a standard users dataset.
There are some special characters in column names that need to be removed and some types that need validation.
I'll output some statistics and a list of unique values in each column to get a better feel for the data.

In [251]:
import pandas as pd

In [252]:
df = pd.read_json('https://raw.githubusercontent.com/chingu-voyages/voyage-project-chingu-map/main/src/assets/chingu_info.json')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8970 entries, 0 to 8969
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Timestamp                     8967 non-null   datetime64[ns]
 1   Gender                        8970 non-null   object        
 2   Country Code                  8970 non-null   object        
 3   Timezone                      8970 non-null   object        
 4   Goal                          8970 non-null   object        
 5   Goal-Other                    8970 non-null   object        
 6   Source                        8970 non-null   object        
 7   Source-Other                  8970 non-null   object        
 8   Country name (from Country)   8970 non-null   object        
 9   Solo Project Tier             8970 non-null   object        
 10  Role Type                     8970 non-null   object        
 11  Voyage Role                   

In [253]:
# display a short list of the unique values in each column
for column in df.columns:
    unique_values = df[column].unique()
    print(f"Column: {column}")
    print(f"Unique values ({len(unique_values)}): {unique_values[:10]}")

Column: Timestamp
Unique values (8929): <DatetimeArray>
['2025-11-05 13:33:00', '2025-11-05 06:52:00', '2025-11-04 09:14:00',
 '2025-11-03 10:40:00', '2025-11-02 08:51:00', '2025-11-01 07:31:00',
 '2025-10-31 14:09:00', '2025-10-31 14:05:00', '2025-10-31 13:34:00',
 '2025-10-31 12:56:00']
Length: 10, dtype: datetime64[ns]
Column: Gender
Unique values (6): ['MALE' 'FEMALE' 'PREFER NOT TO SAY' 'NON-BINARY' 'TRANS' '']
Column: Country Code
Unique values (178): ['NZ' 'IN' 'GE' 'KE' 'IR' 'GB' 'BR' 'CR' 'GH' 'KR']
Column: Timezone
Unique values (34): ['' 'GMT-5 (New York)' 'GMT−5' 'GMT−8' 'GMT+1' 'GMT-5' 'GMT-8' 'GMT+3'
 'GMT+5' 'GMT+10']
Column: Goal
Unique values (6): ['ACCELERATE LEARNING' 'GAIN EXPERIENCE' 'NETWORK WITH SHARED GOALS'
 'GET OUT OF TUTORIAL PURGATORY' 'OTHER' '']
Column: Goal-Other
Unique values (1018): ['' 'Gain real experience and also networking'
 'I like code with people and gain experience'
 'Gain Experience, Collaborating with Team'
 'i want a chance to excercise my 


# Inferred Column Descriptions

| Column | Type | Description |
|---|---|---|
| **Timestamp** | datetime | Signup date for the member submission time (format "YYYY-MM-DD HH:MM") |
| **Gender** | categorical | Gender the chingu selected in their signup form. 'MALE' 'FEMALE' 'PREFER NOT TO SAY' 'NON-BINARY' 'TRANS' |
| **Country Code** | ISO alpha-2-ish | Short country codes for a chingu's country of origin (e.g. US, IN, GB) |
| **Timezone** | categorical | Timezone for which the chingu resides. ("GMT-4", "GMT-5". Input seems to be messy with location sometimes included |
| **Goal** | categorical | Standardized responses for a user wants out of Chingu (GAIN EXPERIENCE, ACCELERATE LEARNING, ...) |
| **Goal-Other** | free text | Supplied elaboration when Goal = OTHER. |
| **Source** | categorical | Standardized responses for how users found Chingu (PERSONAL NETWORK, GOOGLE SEARCH, etc.). |
| **Source-Other** | free text | Additional source detail when Source = OTHER. |
| **Country name (from Country)** | verbose country name | Redundant with Country Code (e.g., "United States", "India"). |
| **Solo Project Tier** | categorical | Tier of project a chingu completed and associated description with that tier |
| **Role Type** | categorical | Type of developer a chingu member is |
| **Voyage Role** | categorical | Role the chingu takes in a Voyage: Developer, Scrum Master, Product Owner, UI/UX Designer. |
| **Voyage (from Voyage Signups)** | semi-structured | List of voyages a chingu has participated in |
| **Voyage Tier** | semi-structured | List of skill levels for the team assigned to in the voyage signup. |


# cleaning plan
- replace special characters in the column names with '_'
- convert empty strings are converted to `NULL` / `None`

- `Goal-Other`, `Source-Other`: Trim whitespace
- `Goal-Other`, `Source-Other`: empty strings are converted to `None`/`NULL`

- add a Serially incrementing `id` column for the primary key 

- `Timestamp`: Convert to UTC - Parse to pandas datetime (coerce errors)

- `Timezone`: messy, coerce into categorical values
- `Timezone`: extract UTC offset tokens (e.g., GMT±n) into a numeric offset column

- `Country name`: some don't match `Country Code`. Use a ISO-3166 alpha-2 mapping to create a new verbose country name col. Report rows that aren't just empty but mismatch between the new and the old verbose `Country name`

- Add arrayed values (`Voyage Signups`,`Voyage Tier`) into separate table: `Voyage_Signups`
- rows in `Voyage_Signups` will be key linked to chingu member row from which they were taken from

- Export the new tables to NDJSON (newline-delimited JSON) for BigQuery ingestion

- TODO: create another jupyternb for clearning the `Voyage_Signups` table

In [254]:
print(df.columns)
df.columns = df.columns.str.replace(pat='Country name (from Country)', repl='Country_name')
df.columns = df.columns.str.replace(pat=r'[^A-Za-z0-9]+', repl='_', regex=True)
df.columns = df.columns.str.strip('_')
print(df.columns)

Index(['Timestamp', 'Gender', 'Country Code', 'Timezone', 'Goal', 'Goal-Other',
       'Source', 'Source-Other', 'Country name (from Country)',
       'Solo Project Tier', 'Role Type', 'Voyage Role',
       'Voyage (from Voyage Signups)', 'Voyage Tier'],
      dtype='object')
Index(['Timestamp', 'Gender', 'Country_Code', 'Timezone', 'Goal', 'Goal_Other',
       'Source', 'Source_Other', 'Country_name', 'Solo_Project_Tier',
       'Role_Type', 'Voyage_Role', 'Voyage_from_Voyage_Signups',
       'Voyage_Tier'],
      dtype='object')


In [255]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8970 entries, 0 to 8969
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Timestamp                   8967 non-null   datetime64[ns]
 1   Gender                      8970 non-null   object        
 2   Country_Code                8970 non-null   object        
 3   Timezone                    8970 non-null   object        
 4   Goal                        8970 non-null   object        
 5   Goal_Other                  8970 non-null   object        
 6   Source                      8970 non-null   object        
 7   Source_Other                8970 non-null   object        
 8   Country_name                8970 non-null   object        
 9   Solo_Project_Tier           8970 non-null   object        
 10  Role_Type                   8970 non-null   object        
 11  Voyage_Role                 8970 non-null   object      

In [256]:
df.replace('', None, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8970 entries, 0 to 8969
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Timestamp                   8967 non-null   datetime64[ns]
 1   Gender                      8969 non-null   object        
 2   Country_Code                8967 non-null   object        
 3   Timezone                    590 non-null    object        
 4   Goal                        8796 non-null   object        
 5   Goal_Other                  1165 non-null   object        
 6   Source                      8849 non-null   object        
 7   Source_Other                3062 non-null   object        
 8   Country_name                7227 non-null   object        
 9   Solo_Project_Tier           1692 non-null   object        
 10  Role_Type                   846 non-null    object        
 11  Voyage_Role                 8470 non-null   object      

In [258]:
# Timestamp: Convert to UTC - Parse to pandas datetime (coerce errors)
df['Timestamp_cleaned'] = pd.to_datetime(df['Timestamp'], errors='coerce').dt.tz_localize('UTC')
df.head()

,Timestamp,Gender,Country_Code,Timezone,Goal,Goal_Other,Source,Source_Other,Country_name,Solo_Project_Tier,Role_Type,Voyage_Role,Voyage_from_Voyage_Signups,Voyage_Tier,id,Timestamp_cleaned
0,2025-11-05 13:33:00,MALE,NZ,None,ACCELERATE LEARNING,None,PERSONAL NETWORK,None,New Zealand,None,Web,Developer,None,None,1,2025-11-05 13:33:00+00:00
1,2025-11-05 06:52:00,FEMALE,IN,None,GAIN EXPERIENCE,None,GOOGLE SEARCH,None,India,None,Python,Developer,None,None,2,2025-11-05 06:52:00+00:00
2,2025-11-04 09:14:00,MALE,GE,None,NETWORK WITH SHARED GOALS,None,GOOGLE SEARCH,None,Georgia,None,Web,Developer,None,None,3,2025-11-04 09:14:00+00:00
3,2025-11-03 10:40:00,MALE,KE,None,ACCELERATE LEARNING,None,The Job Hackers,None,Kenya,None,None,Product Owner,None,None,4,2025-11-03 10:40:00+00:00
4,2025-11-02 08:51:00,FEMALE,IR,None,GAIN EXPERIENCE,None,OTHER,Frontend masters,"Iran, Islamic Republic of",None,Web,Developer,None,None,5,2025-11-02 08:51:00+00:00


In [259]:
free_text_cols = ['Goal_Other', 'Source_Other']
print((df[free_text_cols] == '').sum().sum())
df[free_text_cols] = df[free_text_cols].apply(lambda x: x.str.strip())
df.replace('', None, inplace=True)
print((df[free_text_cols] == None).sum().sum())

0
0


In [260]:
rough_timezones = df['Timezone'].unique()
print(rough_timezones)

[None 'GMT-5 (New York)' 'GMT−5' 'GMT−8' 'GMT+1' 'GMT-5' 'GMT-8' 'GMT+3'
 'GMT+5' 'GMT+10' 'GMT-12' 'GMT+6' 'GMT-6' 'GMT+2' 'GMT+8' 'GMT-0' 'GMT-7'
 'GMT-4' 'GMT+7' 'GMT+9' 'GMT-2' 'GMT-3' 'GMT+4' 'GMT+0' 'GMT+12' 'GMT−7'
 'GMT−6' 'GMT−3' 'GMT−1' 'GMT−10' 'GMT−4' 'GMT+22' '#N/A' 'GMT+)']


In [261]:
def coerce_gmt_offset(offset: int):
    offset = offset%24
    if offset > 12:
        offset -= 24
    elif offset < -11:
        offset += 24

    return offset

offsets =  [25, 24, 23, 13, 12, 11, 5, 1, 0, -1, -5, -11, -12, -13, -20, -23, -24, -25]
expected = [1,  0, -1, -11, 12, 11, 5, 1, 0, -1, -5, -11,  12,  11,   4,   1,   0, -1]

for offset, answer in zip(offsets, expected):
    processed_offset = coerce_gmt_offset(offset)
    assert processed_offset==answer
    # print(f"Match: {processed_offset==answer}, processed offset {processed_offset} : {answer} expected")


In [262]:
# coerce dash characters into 1 type
# replace empty `GMT` into `GMT+0`
# convert values greater than 12 or lower than -11 into appropriately offset values
# coerce all other values into None

import re

def normalize_gmt(tz):
    if tz is None or tz == "":
        return None
    
    # normalize dash types
    tz = re.sub(r"−", "-", tz)

    m = re.match(r"GMT\s*([+-]?\d+)", tz, re.IGNORECASE)
    if not m:
        return None
        
    offset = int(m.group(1))
    offset = coerce_gmt_offset(offset)
    result = (f"GMT+{offset}" if offset >= 0 else f"GMT{offset}")
    return result

rough_timezones = [None, 'GMT-5 (New York)', 'GMT−5', 'GMT−8', 'GMT+1', 'GMT-5', 'GMT-8', 'GMT+3',
 'GMT+5', 'GMT+10', 'GMT-12', 'GMT+6', 'GMT-6', 'GMT+2', 'GMT+8', 'GMT-0', 'GMT-7',
 'GMT-4', 'GMT+7', 'GMT+9', 'GMT-2', 'GMT-3', 'GMT+4', 'GMT+0', 'GMT+12', 'GMT−7',
 'GMT−6', 'GMT−3', 'GMT−1', 'GMT−10', 'GMT−4', 'GMT+22', '#N/A', 'GMT+)']


expected_timezones = [None,'GMT-5','GMT-5', 'GMT-8', 'GMT+1', 'GMT-5', 'GMT-8', 'GMT+3',
 'GMT+5', 'GMT+10', 'GMT+12', 'GMT+6', 'GMT-6', 'GMT+2', 'GMT+8', 'GMT+0', 'GMT-7',
 'GMT-4', 'GMT+7', 'GMT+9', 'GMT-2', 'GMT-3', 'GMT+4', 'GMT+0', 'GMT+12', 'GMT-7',
 'GMT-6', 'GMT-3', 'GMT-1', 'GMT-10', 'GMT-4', 'GMT-2', None, None]
for rough_tz, answer_tz  in zip(rough_timezones, expected_timezones):
    processed_tz = normalize_gmt(rough_tz)
    assert processed_offset==answer
    # print(rough_tz, processed_tz, answer_tz, processed_tz==answer_tz)


In [263]:
before = 'Timezone'
after = 'Timezone_cleaned'

df['Timezone_cleaned'] = df['Timezone'].map(normalize_gmt)

mask = (df[before] != df[after]) & ~(df[before].isna() & df[after].isna())
df.loc[mask, [before, after]].drop_duplicates()

,Timezone,Timezone_cleaned
5627,GMT-5 (New York),GMT-5
7915,GMT−5,GMT-5
8266,GMT−8,GMT-8
8394,GMT-12,GMT+12
8424,GMT-0,GMT+0
8598,GMT−7,GMT-7
8600,GMT−6,GMT-6
8627,GMT−3,GMT-3
8647,GMT−1,GMT-1
8654,GMT−10,GMT-10


In [264]:
df_filtered = df[['Country_Code', 'Country_name']].drop_duplicates().groupby('Country_Code').filter(func=lambda x: len(x) > 1, dropna=True)

df_filtered.sort_values('Country_Code')

,Country_Code,Country_name
305,AE,United Arab Emirates
6596,AE,None
1886,AI,Anguilla
8491,AI,None
327,AM,Armenia
...,...,...
6544,ZA,None
1614,ZM,Zambia
7249,ZM,None
6440,ZW,None


In [265]:
# Find Country Codes mapping to NOT ONLY None values but multiple Country names
empty_country_name_filter = df['Country_name'].isnull()
df_filtered = df[empty_country_name_filter != True]
df_filtered = df_filtered[['Country_Code', 'Country_name']].drop_duplicates().groupby('Country_Code').filter(func=lambda x: len(x) > 1, dropna=True)

df_filtered.sort_values('Country_Code')

,Country_Code,Country_name
91,IT,Italy
2718,IT,Germany
13,US,United States
5303,US,Netherlands


# Comments on `Country_Code` and `Country_name`
The previous 2 cells demonstrate that `Country_Code` not only maps to `None` values but also incorrect values. Earlier when we removed empty string values from all of the data, it showed that the `Country_Code` column was 99% complete. I think it's reasonable to assume it's validity over the `Country_name` column entirely and use a library and fill the `Country_name` column using `Country_Code`.


# 

In [ ]:
import country_converter as coco
cc = coco.CountryConverter()

In [268]:
df['Country_Name_cleaned'] = cc.convert(
    df['Country_Code'],
    to='name_short',
    not_found=None
)

None not found in regex
None not found in regex
UT not found in ISO2
None not found in regex


In [273]:
df['Country_Code'].unique()

array(['NZ', 'IN', 'GE', 'KE', 'IR', 'GB', 'BR', 'CR', 'GH', 'KR', 'CA',
       'US', 'SA', 'ID', 'FR', 'ES', 'NL', 'ZA', 'NG', 'DZ', 'MA', 'DE',
       'PL', 'HU', 'AR', 'TH', 'IT', 'JM', 'DK', 'VN', 'NP', 'JP', 'UA',
       'UG', 'AU', 'PH', 'SE', 'IQ', 'TW', 'PK', 'GR', 'EG', 'LB', 'EC',
       'TZ', 'CO', 'SD', 'KG', 'RS', 'MG', 'ET', 'RU', 'NI', 'BY', 'HK',
       'ZW', 'MX', 'BD', 'LV', 'QA', 'SY', 'AE', 'LK', 'CD', 'TR', 'KW',
       'AM', 'SK', 'SG', 'BH', 'RO', 'PS', 'IE', 'CM', 'UZ', 'KH', 'IL',
       'LY', 'AO', 'AF', 'MY', 'CL', 'TN', 'CN', 'BG', 'PT', 'RW', 'BW',
       'BE', 'LS', 'JO', 'LT', 'MV', 'CZ', 'CH', 'TT', 'MW', 'MD', 'PE',
       'AG', 'MM', 'AZ', 'VI', 'AL', 'DO', 'YE', 'AT', 'FI', 'NO', 'ZM',
       'CX', 'UY', 'CF', 'LA', 'AI', 'AQ', 'GY', 'SV', 'MZ', 'NA', 'MU',
       'VE', 'KI', 'BN', 'HR', 'BA', 'PG', 'SL', 'PR', 'GN', 'EE', 'BJ',
       'KZ', 'HT', 'BT', 'GI', 'TJ', 'AS', 'DJ', 'TM', 'MR', 'SO', 'OM',
       'FO', 'LU', 'SN', 'BO', 'MN', 'TG', 'GT', 'C

In [284]:
PH_filter = df['Country_Code']=='Philippines (PH)'
UT_filter = df['Country_Code']=='UT'
print(df[PH_filter | UT_filter][['Country_Code', 'Country_name']])

          Country_Code Country_name
5982  Philippines (PH)  Philippines
8964                UT         None


In [ ]:
# errors in Country Code are so few, we can clean them manually
df['Country_Code_cleaned'] = (
    df['Country_Code']
      .replace({'UT': None})
)

In [ ]:
before = 'Country_name'
after = 'Country_Name_cleaned'

mask = (df[before] != df[after]) & ~(df[before].isna() & df[after].isna())
df.loc[mask, ['Country_Code_cleaned', before, after]].drop_duplicates()

,Country_Code_cleaned,Country_name,Country_Name_cleaned
4,IR,"Iran, Islamic Republic of",Iran
11,KR,"Korea, Republic of",South Korea
148,TW,"Taiwan, Province of China",Taiwan
186,TZ,"Tanzania, United Republic of",Tanzania
222,RS,Republic of Serbia,Serbia
...,...,...,...
8541,AZ,None,Azerbaijan
8556,JO,None,Jordan
8940,SK,None,Slovakia
8961,None,None,None


In [ ]:
df_grouped = df.groupby(['Voyage_Role', 'Role_Type'], dropna=False).size()
print(df_grouped)

Voyage_Role     Role_Type
Data Scientist  NaN           654
Developer       Python        194
                Web           652
                NaN          5433
Product Owner   NaN           694
Scrum Master    NaN           265
UI/UX Designer  NaN           578
NaN             NaN           500
dtype: int64


In [291]:
# df['Role_agg'] = df[['Role_Type', 'Voyage_Role']].agg(lambda x: ' '.join(x.dropna()), axis='columns')
df['Role'] = df['Voyage_Role'].str.cat(df['Role_Type'], sep=' ', na_rep='')
df['Role'].value_counts(dropna=False)

Role
Developer           5433
Product Owner        694
Data Scientist       654
Developer Web        652
UI/UX Designer       578
                     500
Scrum Master         265
Developer Python     194
Name: count, dtype: int64

In [292]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8970 entries, 0 to 8969
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   Timestamp                   8967 non-null   datetime64[ns]     
 1   Gender                      8969 non-null   object             
 2   Country_Code                8967 non-null   object             
 3   Timezone                    590 non-null    object             
 4   Goal                        8796 non-null   object             
 5   Goal_Other                  1163 non-null   object             
 6   Source                      8849 non-null   object             
 7   Source_Other                3060 non-null   object             
 8   Country_name                7227 non-null   object             
 9   Solo_Project_Tier           1692 non-null   object             
 10  Role_Type                   846 non-null    object          

In [ ]:
df['id'] = range(1, len(df) + 1)
df.head()

In [ ]:
# drop old or irrelevant columns
# cleaned columns
[
    'Timestamp',
    'Timezone',
    'Country_name',
]

# irrelevant columns
[
    'Role_Type',
    'Voyage_Role',
    'Voyage_from_Voyage_Signups',
    'Voyage_Tier',
]

In [ ]:
# give columns their final names
{
    'Timestamp_cleaned': 'Timestamp',
    'Timezone_cleaned': 'Timezone',
    'Country_Name_cleaned': 'Country_Name',
}

In [ ]:
# write cleaned records
# with open("data/chingu_members_cleaned.json", "w") as f:
#     for record in df:
#         json.dump(record, f)
#         f.write("\n")